In [32]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import urllib
import re
from IPython.display import display, HTML
import HTMLParser

for pageno in range(201):

    r = urllib.urlopen('https://www.kickstarter.com/discover/advanced?woe_id=0&sort=magic&seed=2462842&page='+str(pageno+1)).read()
    soup = BeautifulSoup(r, 'html.parser')

    proj_title = soup.find_all("h6", class_="project-title")

    # === Project descriptions ===
    title_str = HTMLParser.HTMLParser().unescape(str(proj_title)) # remove the &quot
    proj_title_link = re.findall('href="(/projects/.*?)"', title_str)
    proj_title_title = [kk.string for kk in proj_title]
    proj_byline = soup.find_all("p", class_="project-byline")
    proj_byline_title = [kk.string for kk in proj_byline]
    proj_blurb = soup.find_all("p", class_="project-blurb")
    proj_blurb_title = [kk.string for kk in proj_blurb]    
    
    # === Project Location ===
    proj_location = soup.find_all("div", class_="project-location")
    proj_location_str = HTMLParser.HTMLParser().unescape(str(proj_location)) # remove the &quot
    proj_location_shortname = re.findall('"displayable_name":"(.*?)"', proj_location_str)
    proj_location_country = re.findall('"country":"(.*?)"', proj_location_str)
    proj_location_state = re.findall('"state":"(.*?)"', proj_location_str)
    proj_location_type = re.findall('"type":"(.*?)"', proj_location_str)
    
#     print proj_location_str
#     print proj_location_shortname
#     print proj_location_country
#     print proj_location_state
#     print proj_location_type

    # === Project Stat ===
    proj_stat = soup.find_all("ul", class_="project-stats")

    percent_funded_list=[]
    currency_type_list = []
    amt_pledged_list = []
    dollar_sign_list = []
    amount_list = []
    data_end_time_list = []
    

   
    for i_string in proj_stat:
        proj_stat_str = HTMLParser.HTMLParser().unescape(str(i_string))
#         print 'proj_stat_str'
#         print proj_stat_str
        percent_funded = re.findall('<div class="project-stats-value">(.*?)%</div>', proj_stat_str)[0]
        percent_funded_list.append(percent_funded[0])
        whole_dollar = re.findall('<div class="project-stats-value"><span class="money">(.*?)</span></div>', proj_stat_str)[0]
        whole_dollar_split = re.split('(\d+)',whole_dollar)
#         print whole_dollar_split
        dollar_sign_list.append( whole_dollar_split[0] )
        
        whole_dollar_split[0] = ''
        whole_dollar_split = [ss.replace(',','') for ss in whole_dollar_split]
        amount_list.append(float(''.join(whole_dollar_split)))
        data_end_time_list.append( re.findall('<li class="ksr_page_timer" data-end_time=(.*?)>', proj_stat_str)[0] )
        
        
#         print (''.join(whole_dollar_split))
#         print (int(''.join(whole_dollar_split)))

        
        #         print 'whole_dollar'
#         print whole_dollar
#         print 'dollar sign'
#         print whole_dollar[0]
#         print 'amount'
#         print whole_dollar[1:]
        
    
    # ==== This was the old code. Not useful ===
#         currency_type = re.findall('<span class="money (.*?) no-code">', proj_stat_str)
# #         currency_type_list.append(currency_type[0])
#         print 'currency_type'
#         print currency_type

#         amt_pledged = re.findall('no-code">(.*?)</span>', proj_stat_str)
#         amt_pledged_str = str(amt_pledged[0])
#         amt_pledged_list.append( amt_pledged_str )

#         <ul class="project-stats">
# <li>
# <div class="project-stats-value">76%</div> <span class="project-stats-label">funded</span>
# </li>
# <li class="js-amount-pledged">
# <div class="project-stats-value"><span class="money">$76,439</span></div> <span class="project-stats-label">pledged</span>
# </li>
# <li style="display: none;">
# <div class="project-stats-value">Funded</div>
# <span class="project-stats-label">
# </span>
# </li>
# <li class="ksr_page_timer" data-end_time="2017-03-28T07:58:00-04:00">
# <div class="project-stats-value">
# <div class="js-num"> </div>
# </div>
# <div class="project-stats-label js-text" data-word="left"> </div>
# </li>
# </ul>
    

    # === This is for debugging ===
#     print proj_location_str
#     print len(proj_title_title)
#     print proj_title_title
#     print len(proj_title_link)
#     print proj_location_shortname
#     print len(proj_location_shortname)
#     print len(proj_blurb_title)
#     print len(proj_location_country)
#     print len(proj_location_state) 

#     print len(proj_location_type) 
#     print len(percent_funded_list)
#     print len(currency_type_list)
#     print len(amt_pledged_list)

    # === Put them into data frame ===
    try:
        df = pd.DataFrame({'Project title':proj_title_title, 'Project Link':proj_title_link,\
                           'Project byline':proj_byline_title, 'Project blurb':proj_blurb_title,\
                           'Location':proj_location_shortname, 'Country': proj_location_country,\
                          'State': proj_location_state, 'Type':proj_location_type, \
                          'Percentage funded':percent_funded_list,\
                          'Dollar sign': dollar_sign_list, 'Amount':amount_list, 'data_end_time_list':data_end_time_list})
    except:
        print 'Page #'+str(pageno+1)+' has an exception.'
        break

    if pageno==0:
        df2 = df
    else:
        df2 = pd.concat([df2,df])
        
# === Reset index such that it is monotonic ===        
df2 = df2.reset_index(drop=True)
df2.info()
display(df2)


Page #177 has an exception.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3520 entries, 0 to 3519
Data columns (total 12 columns):
Amount                3520 non-null float64
Country               3520 non-null object
Dollar sign           3520 non-null object
Location              3520 non-null object
Percentage funded     3520 non-null object
Project Link          3520 non-null object
Project blurb         3520 non-null object
Project byline        3520 non-null object
Project title         3520 non-null object
State                 3520 non-null object
Type                  3520 non-null object
data_end_time_list    3520 non-null object
dtypes: float64(1), object(11)
memory usage: 330.1+ KB


,Amount,Country,Dollar sign,Location,Percentage funded,Project Link,Project blurb,Project byline,Project title,State,Type,data_end_time_list
0,76439.0,US,$,"Los Angeles, CA",7,/projects/blackholes/black-holes-0?ref=discovery,"An adult animated series about Dave, a mars-b...",Noodles Studio,Black Holes,CA,Town,"""2017-03-28T07:58:00-04:00"""
1,5539.0,SE,$,"Malm\xf6, Sweden",2,/projects/129924971/dead-donkeys-fear-no-hyena...,"After a 7-year long journey, we have a comple...",WG Film,Dead Donkeys Fear No Hyenas - A feature-length...,Skane,Town,"""2017-04-12T13:00:00-04:00"""
2,53423.0,US,$,"Fairplay, CO",4,/projects/1402759381/buffalo-peaks-ranch-a-lit...,Help transform an old cattle ranch in Colorad...,Rocky Mountain Land Library,The Rocky Mountain Land Library,CO,Town,"""2017-04-08T01:59:00-04:00"""
3,298440.0,US,$,"San Diego, CA",1,/projects/rifftrax/rifftrax-live-17-samurai-co...,RiffTrax is doing three live shows in 2017 si...,"RiffTrax: Mike, Bill & Kevin","RiffTrax Live '17: Samurai Cop, Beach Party & ...",CA,Town,"""2017-03-25T21:00:00-04:00"""
4,21635.0,GB,£,"Liverpool, UK",8,/projects/swapbots/swapbots-the-augmented-real...,Bring these collectible toys to life with you...,SwapBots,SwapBots: Augmented Reality Toys For Creative ...,England,Town,"""2017-04-12T11:57:53-04:00"""
5,58990.0,US,$,"New York, NY",1,/projects/1289830263/nowadays-help-us-build-ou...,After almost a decade of growing a community ...,Mister Saturday Night,NOWADAYS: HELP US BUILD OUR INDOOR HOME,NY,Town,"""2017-04-05T17:00:00-04:00"""
6,8436.0,US,$,"Atlanta, GA",5,/projects/oghmc/she-persisted-enamel-pin-color...,"I want to expand the color palette of my ""She...",Holly M. Clifton,"""She Persisted"" Enamel Pin: Color Palette Expa...",GA,Town,"""2017-04-17T03:00:00-04:00"""
7,43137.0,US,$,"New York, NY",2,/projects/impellerstudios/starfighter-inc-0?re...,Online multiplayer space combat simulator bas...,Impeller Studios,STARFIGHTER INC,NY,Town,"""2017-04-15T13:50:54-04:00"""
8,3170.0,DE,$,"Berlin, Germany",2,/projects/adamhurly/the-prospectives-a-narrati...,"For its fourth installment, the serialized In...",Adam Hurly,The Prospectives: A Narrative Podcast,Berlin,Town,"""2017-04-16T16:00:00-04:00"""
9,14390.0,GB,£,"Greater London, UK",1,/projects/1412864360/you-are-the-hero-part-2?r...,"A supplement to 2014's YOU ARE THE HERO, to m...",Jonathan Green,YOU ARE THE HERO Part 2,England,County,"""2017-04-01T08:00:00-04:00"""


## Dollar-sign converter
http://utf8-chartable.de/unicode-utf8-table.pl?start=8320&number=128&names=-&utf8=string-literal

## Positive and Negative word List
Tutorial: http://nealcaren.web.unc.edu/pizza-twitter-and-apis

Positive List: http://www.unc.edu/~ncaren/haphazard/positive.txt        

Negative List: http://www.unc.edu/~ncaren/haphazard/negative.txt

In [39]:
print df2.columns
# Index([u'Amount', u'Country', u'Dollar sign', u'Location',
#        u'Percentage funded', u'Project Link', u'Project blurb',
#        u'Project byline', u'Project title', u'State', u'Type',
#        u'data_end_time_list'],
#       dtype='object')    

for col in df2.columns:
    df2.to_csv(col+'.csv',columns = [col],encoding = 'utf-8')
    


Index([u'Amount', u'Country', u'Dollar sign', u'Location',
       u'Percentage funded', u'Project Link', u'Project blurb',
       u'Project byline', u'Project title', u'State', u'Type',
       u'data_end_time_list'],
      dtype='object')


In [38]:
df2

,Amount,Country,Dollar sign,Location,Percentage funded,Project Link,Project blurb,Project byline,Project title,State,Type,data_end_time_list
0,76439.0,US,$,"Los Angeles, CA",7,/projects/blackholes/black-holes-0?ref=discovery,"An adult animated series about Dave, a mars-b...",Noodles Studio,Black Holes,CA,Town,"""2017-03-28T07:58:00-04:00"""
1,5539.0,SE,$,"Malm\xf6, Sweden",2,/projects/129924971/dead-donkeys-fear-no-hyena...,"After a 7-year long journey, we have a comple...",WG Film,Dead Donkeys Fear No Hyenas - A feature-length...,Skane,Town,"""2017-04-12T13:00:00-04:00"""
2,53423.0,US,$,"Fairplay, CO",4,/projects/1402759381/buffalo-peaks-ranch-a-lit...,Help transform an old cattle ranch in Colorad...,Rocky Mountain Land Library,The Rocky Mountain Land Library,CO,Town,"""2017-04-08T01:59:00-04:00"""
3,298440.0,US,$,"San Diego, CA",1,/projects/rifftrax/rifftrax-live-17-samurai-co...,RiffTrax is doing three live shows in 2017 si...,"RiffTrax: Mike, Bill & Kevin","RiffTrax Live '17: Samurai Cop, Beach Party & ...",CA,Town,"""2017-03-25T21:00:00-04:00"""
4,21635.0,GB,£,"Liverpool, UK",8,/projects/swapbots/swapbots-the-augmented-real...,Bring these collectible toys to life with you...,SwapBots,SwapBots: Augmented Reality Toys For Creative ...,England,Town,"""2017-04-12T11:57:53-04:00"""
5,58990.0,US,$,"New York, NY",1,/projects/1289830263/nowadays-help-us-build-ou...,After almost a decade of growing a community ...,Mister Saturday Night,NOWADAYS: HELP US BUILD OUR INDOOR HOME,NY,Town,"""2017-04-05T17:00:00-04:00"""
6,8436.0,US,$,"Atlanta, GA",5,/projects/oghmc/she-persisted-enamel-pin-color...,"I want to expand the color palette of my ""She...",Holly M. Clifton,"""She Persisted"" Enamel Pin: Color Palette Expa...",GA,Town,"""2017-04-17T03:00:00-04:00"""
7,43137.0,US,$,"New York, NY",2,/projects/impellerstudios/starfighter-inc-0?re...,Online multiplayer space combat simulator bas...,Impeller Studios,STARFIGHTER INC,NY,Town,"""2017-04-15T13:50:54-04:00"""
8,3170.0,DE,$,"Berlin, Germany",2,/projects/adamhurly/the-prospectives-a-narrati...,"For its fourth installment, the serialized In...",Adam Hurly,The Prospectives: A Narrative Podcast,Berlin,Town,"""2017-04-16T16:00:00-04:00"""
9,14390.0,GB,£,"Greater London, UK",1,/projects/1412864360/you-are-the-hero-part-2?r...,"A supplement to 2014's YOU ARE THE HERO, to m...",Jonathan Green,YOU ARE THE HERO Part 2,England,County,"""2017-04-01T08:00:00-04:00"""
